In [1]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import os
import tqdm

from IDAO_validator_2d_model import IDAO_validator_2d_model

In [2]:
def smape(satellite_predicted_values, satellite_true_values): 
    # the division, addition and subtraction are pointwise 
    return np.mean(np.abs((satellite_predicted_values - satellite_true_values) 
        / (np.abs(satellite_predicted_values) + np.abs(satellite_true_values))))

In [3]:
# Загрузка данных
# windows
PATH_TO_DATA = os.path.join('../data')
full_train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'), index_col='id')
full_test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'), index_col='id')

In [4]:
idao_2d_splitter = IDAO_validator_2d_model(5, 5)

In [5]:
import lightgbm

In [6]:
%%time

## колонки, которые не нужны для предсказания
columns_to_drop = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz', 'epoch', 'sat_id']

## целевые колонки
target_columns = ['x', 'y', 'z', 'Vx', 'Vy', 'Vz']

## номер фолда для вывода логов
n_fold = 0
## предыдущий 'sat_id' - опять же для вывода логов - чтобы отслеживать смену объектов (sat_id)
prev_sat_id = 0

## список из значений smape (на тесте) для каждого из sat_id
smape_per_sat_id = []

for trn_idxs, tst_idxs in idao_2d_splitter.split(full_train):
    
    ## для каждого sat_id обновляем n_fold (для логов)
    for sat_id in full_train['sat_id'].unique():
        ## создаем массив для хранения предсказаний для теста, чтобы посчитать smape
        preds = np.zeros((tst_idxs[sat_id, 1] - tst_idxs[sat_id, 0], len(target_columns)))
        ## массив для хранения реальных значений целевых переменных для теста
        y_true = np.zeros((tst_idxs[sat_id, 1] - tst_idxs[sat_id, 0], len(target_columns)))
        ## цикл по целевым колонкам
        for i, col in enumerate(target_columns):

            lgbm = lightgbm.LGBMRegressor(max_depth=5, learning_rate=0.2, n_estimators=1000, reg_alpha=5e-6,
                                          colsample_bytree = 0.85,
                                          reg_lambda=5e-4,
                                          random_state=13)
            X_train = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]].drop(columns=columns_to_drop).values
            y_train = full_train.iloc[trn_idxs[sat_id, 0]:trn_idxs[sat_id, 1]][col].values

            X_test = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]].drop(columns=columns_to_drop).values
            y_test = full_train.iloc[tst_idxs[sat_id, 0]:tst_idxs[sat_id, 1]][col].values
            if X_test.shape[0] != 0:
                lgbm.fit(X_train, y_train)

                preds[:, i] += lgbm.predict(X_test)
                y_true[:, i] = y_test

            ## выводим метрику для каждой целевой колонки отдельно
            print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: {col},\
                  smape: {1 - smape(preds[:, i], y_test):.5f}')
            
        ## выводим метрику для всех целевых колонок сразу (для одного sat_id)
        print('_' * 15)
        print(f'sat_id: {sat_id}, fold: {n_fold}, target_col: ALL_COLUMNS,\
                  smape: {1 - smape(preds, y_true):.5f}')

        smape_per_sat_id.append(smape(preds, y_true))
        print('_' * 15)
    n_fold += 1

sat_id: 0, fold: 0, target_col: x,                  smape: 0.85068
sat_id: 0, fold: 0, target_col: y,                  smape: 0.85312
sat_id: 0, fold: 0, target_col: z,                  smape: 0.84829
sat_id: 0, fold: 0, target_col: Vx,                  smape: 0.85590
sat_id: 0, fold: 0, target_col: Vy,                  smape: 0.82734
sat_id: 0, fold: 0, target_col: Vz,                  smape: 0.85745
_______________
sat_id: 0, fold: 0, target_col: ALL_COLUMNS,                  smape: 0.84880
_______________


KeyboardInterrupt: 

In [12]:
print(1 - np.mean(smape_per_sat_id))

0.8961579202144785


0.8961579202144785